In [79]:
import sys
import struct
from PIL import Image
def readDataFromDDF(ddf):
    with open(ddf,'rb') as f:
        channels = 3
        
        #get defect number
        f.seek(80, 0)
        defs = int.from_bytes(f.read(1), byteorder='big')
        print("Defect Count: ", defs)
        if defs <= 0:
            print("Broken file to get defects number")
            return
        
        #get defect size
        f.seek(0, 0)
        imgM = f.read(4).decode("utf-8") 
        if imgM == "HMIU":
            imgw = 32
        elif imgM == "HMIV":
            imgw = 64
        elif imgM == "HMIW":
            imgw = 128
        elif imgM == "HMIX":
            imgw = 192
        elif imgM == "HMIY":
            imgw = 256
        elif imgM == "HMIZ":
            imgw = 512
        print("Defect Size: ", imgw)
        
        #
        save_to_img = True
        buff = []
        f.seek(1024, 0)#skip 1k head
        for imgIdx in range(defs):
            seg = []
            for ichannel in range(channels):
                piece = []
                for ipixel in range(imgw * imgw):
                    piece.append(int.from_bytes(f.read(1), byteorder='big'))
                
                if save_to_img:
                    new_img = Image.new("L", (imgw, imgw), "white")
                    new_img.putdata(piece)
                    new_img.save('data/%d_%d_def.jpg' % (imgIdx, ichannel))
                else:
                    seg.append(piece)
            if not save_to_img:
                buff.append(seg)
    return
        

In [80]:
cont = readDataFromDDF("./SetLine.ddf")

Defect Count:  130
Defect Size:  64


In [18]:
import xml.etree.ElementTree as ET
def readLabelFromDZF(dzf):
    root = ET.parse(dzf).getroot()
    #print(root.tag)
    #print(root.attrib)
    
    defs = root.find("Test").find('DefectList')
    defslist = []
    for d in defs.findall('Defect'):
        defslist.append([d.attrib['Index'], d.attrib['ImageID'],d.attrib['ReviewType']])
    
    #print(defslist)
    with open('defectList.md', 'w') as handler:
        handler.writelines("%s\n" % df for df in defslist)
    return

In [19]:
info = readLabelFromDZF("./SetLine.dzf")
